# COMPETENCIAS


Librerias externas necesarias para el scraping

In [1]:
#pip install selenium webdriver-manager
#pip install undetected-chromedriver
#pip install bs4
#pip install pandas
#pip install playwright
#pip install playwright-stealth
#playwright install
#!pip install dotenv

Importar librerias que utilizarán durante todo el codigo

In [1]:
import re
import csv
import datetime
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import csv, datetime, time
from bs4 import BeautifulSoup
import time
from collections import Counter
import requests
import pickle
import pandas as pd
from datetime import datetime
import os

URL necesarias para el trabajo del scraping, a demas del final del bloque que contiene configuración basica de Selenium

In [ ]:


# ===== CONFIG =====
OUTPUT_CSV = "yapo_filters.csv"    # archivo resultante
WRITE_HEADER = True                # poner True la primera vez, luego False si quieres append sin header
URL = "https://www.yapo.cl/autos-usados"
URLml = "https://vehiculos.mercadolibre.cl/"
URL_MACAL = "https://www.macal.cl/autos"
URL_AUTOPIA = "https://autopia.cl/autos-usados"
URL_LINK = "https://www.link.cl/vehiculos"
URL_KAVAK = "https://www.kavak.com/cl/usados"
URL_CLICAR = "https://www.clicar.cl/vehiculos/usado"
URLDILY = "https://dilyautos.cl/resultados"
OUTPUT_FILE = "dilyautos_vehiculos.csv"
URL_BRAVO = "https://bravoauto.cl/autos"
URL_AUTOCL = "https://www.auto.cl/todos-los-autos"
URLVenpu = "https://venpu.cl/vehiculos"
URL = "https://www.chilemotos.com/moto/"
BASE_URL = "https://udach.cl/catalogo"
URL_UDACH = "https://udach.cl/catalogo?page={}"
URL_AUTOSUSADOS = "https://autosusados.cl/vehiculos"
URL_CHILEMOTO = "https://www.chilemotos.com/moto/"
URL_AUTOSCOSMO = "https://www.autocosmos.cl/auto"
URLYAPO = "https://www.yapo.cl/chile/autos"  # 👈 ajusta si usas otro


# Configuracicion básica de Selenium
# ===== Selenium (headless) =====
options = Options()
#options.add_argument("--headless=new")   # headless moderno
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1280,2000")



Acá comienza el codigo del scrapeo cómo tal, en este bloque especifico comenzamos a extraer datos de Yapo, de ahí tomamos el numero de autos totales publicados en su plataforma. A demas traerá y guardará en el CSV 'competencia' los filtros y su cantidad de autos registrados de cada cual, ejemplo: auto: 100, camionetas: 120....

In [4]:

# ===== CONFIG =====
OUTPUT_CSV = "vehiculos_totales.csv"
COMPETENCIA_CSV = "competencia.csv"
URL_YAPO = "https://www.yapo.cl/chile/autos"  # 👈 ajusta si usas otro

options = Options()
options.add_argument("--headless=new")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1280,2000")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 20)

try:
    driver.get(URL_YAPO)

    # Esperar a que carguen los labels de filtros
    labels = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.d3-checkbox label")))

    filas = []
    #now = datetime.datetime.now().isoformat(timespec="seconds")
    now = datetime.now().isoformat(timespec="seconds")
    total_vehiculos = 0

    for lb in labels:
        texto = lb.text.strip()
        if not texto:
            continue
        # Extraer número entre paréntesis
        m = re.search(r"\(([\d\.]+)\)", texto)
        numero = int(m.group(1).replace(".", "")) if m else 0

        # Nombre de la categoría
        nombre = re.sub(r"\s*\([\d\.\)]+\)\s*$", "", texto).strip()
        nombre_ant = nombre
        if nombre == "Autos Usados":
            nombre = "Autos y camionetas"
        else: nombre = nombre_ant
    

        # Sumar total excluyendo "Yo busco"
        if nombre.lower() != "yo busco":
            total_vehiculos += numero

        filas.append({
            "timestamp": now,
            "competencia": "yapo",
            "categoria": nombre,
            "cantidad": numero
        })

    # === Guardar detalle de categorías en competencia.csv ===
    file_exists_comp = os.path.exists(COMPETENCIA_CSV)
    with open(COMPETENCIA_CSV, "a", newline="", encoding="utf-8") as f:
        fieldnames_comp = ["timestamp", "competencia" ,"categoria", "cantidad"]
        writer = csv.DictWriter(f, fieldnames=fieldnames_comp)
        if not file_exists_comp:
            writer.writeheader()
        writer.writerows(filas)

    print(f"✅ Guardadas {len(filas)} filas en {COMPETENCIA_CSV}")

    # === Guardar el total en vehiculos_totales.csv ===
    file_exists_main = os.path.exists(OUTPUT_CSV)
    with open(OUTPUT_CSV, "a", newline="", encoding="utf-8") as f:
        fieldnames_main = ["timestamp", "url", "total_vehiculos"]
        writer = csv.DictWriter(f, fieldnames=fieldnames_main)
        if not file_exists_main:
            writer.writeheader()
        writer.writerow({
            "timestamp": now,
            "url": "Yapo",
            "total_vehiculos": total_vehiculos
        })

    print(f"✅ Total agregado ({total_vehiculos}) a {OUTPUT_CSV}")

finally:
    driver.quit()


✅ Guardadas 7 filas en competencia.csv
✅ Total agregado (47158) a vehiculos_totales.csv


Aquí extraemos los datos de Mercadolibre uno de los competidores mas fuertes, este codigo es muy complejo ya qué ML necesita cookies necesarias, las cuales puedes sacar con Selenium... Recomiendo pedirle a CHATGPT que te pase un codigo para guardar cookies al iniciar sesion. Luego de eso obtiene el total de autos de la web, a demás para el CSV 'competencia' mapeamos las categorias que tiene  mercadolibre y las agrupamos como Yapo. Eso nos servirá para tener información homogenea.

In [ ]:
from datetime import datetime

# --- Configuración ---
CSV_EXISTENTE = "competencia.csv"
URLML = "https://autos.mercadolibre.cl/"

# Mapeo de categorías válidas
categorias_map = {
    "Autos y camionetas": [
        "SUV", "Hatchback", "Sedán", "Pick-Up", "Van",
         "Coupé", "Cabriolet", "Crossover",
        "Monovolumen", "Off-Road", "Roadster", "Rural", "Eléctrico", "Híbrido", "4x4"
    ],
    "Camiones y Buses": ["Furgón", "Minivan"],
    "Motos": ["Motocicleta", "Scooters eléctricos"],
    "Maquinaria pesada": ["Light Truck"],
    "Autos Nuevos": ["Nuevo"],
    "Arriendo de Autos": [],
    "Yo busco": []
}

# --- Categorías que no aparecen en el filtro ---
extra_urls = {
    "Crossover": "https://autos.mercadolibre.cl/crossover/",
    "Monovolumen": "https://autos.mercadolibre.cl/monovolumen/",
    "Rural": "https://autos.mercadolibre.cl/rural/",
    "Off-Road": "https://autos.mercadolibre.cl/off-road/",
    "Roadster": "https://autos.mercadolibre.cl/roadster/",
    "Scooters eléctricos": "https://motos.mercadolibre.cl/scooters-electricos",
    "Motocicleta": "https://motos.mercadolibre.cl/motos",
    "Eléctrico": "https://autos.mercadolibre.cl/electrico/",
    "Híbrido": "https://autos.mercadolibre.cl/hibrido",
    "4x4": "https://autos.mercadolibre.cl/4x4",
}

# --- Conexión con cookies ---
cookies = pickle.load(open("cookies.pkl", "rb"))
session = requests.Session()
for cookie in cookies:
    session.cookies.set(cookie['name'], cookie['value'])

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/114.0.0.0 Safari/537.36",
    "Accept-Language": "es-CL,es;q=0.9,en;q=0.8"
}

categorias = []

# --- Scraping principal ---
response = session.get(URLML, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

# Extraer total de vehículos
total_resultados = soup.select_one("span.ui-search-search-result__quantity-results")
if total_resultados:
    total_vehiculos = int(re.sub(r"[^\d]", "", total_resultados.get_text(strip=True)))
else:
    total_vehiculos = 0

# Categorías de la página
for a in soup.select("div.ui-search-filter-dl ul li a.ui-search-link"):
    label_tag = a.select_one("span.ui-search-filter-name")
    count_tag = a.select_one("span.ui-search-filter-results-qty")

    if not label_tag or not count_tag:
        continue

    label = label_tag.get_text(strip=True)
    count_text = count_tag.get_text(strip=True)
    count = int(re.sub(r"[^\d]", "", count_text))
    url = a.get("href")

    if any(label in subs for subs in categorias_map.values()):
        grupo = next((cat for cat, subs in categorias_map.items() if label in subs), "Otros")
        
        categorias.append({
            "timestamp": datetime.now().isoformat(),
            "categoria": grupo,
            "competencia": "Mercadolibre",
            "cantidad": count,
            })

# --- Scraping de categorías extra ---
for label, url in extra_urls.items():
    resp = session.get(url, headers=headers)
    extra_soup = BeautifulSoup(resp.text, "html.parser")
    count_tag = extra_soup.select_one("span.ui-search-search-result__quantity-results")
    
    if not count_tag:
        continue
    
    count = int(re.sub(r"[^\d]", "", count_tag.get_text(strip=True)))
    grupo = next((cat for cat, subs in categorias_map.items() if label in subs), "Otros")

    categorias.append({
        "timestamp": datetime.now().isoformat(),
        "categoria": grupo,
        "competencia": "Mercadolibre",
        "cantidad": count,
    })


# --- Ajustar totales ---
suma_categorias = sum(item["cantidad"] for item in categorias)
diferencia = total_vehiculos - suma_categorias

if diferencia != 0:
    categorias.append({
        "timestamp": datetime.now().isoformat(),
        "categoria": "Autos sin categoría",
        "cantidad": diferencia,
        "url": URLML
    })

# --- Guardar en CSV ---
df = pd.DataFrame(categorias)


# --- Agrupar por categoria y competencia para sumar la cantidad ---
df_agrupado = df.groupby(["categoria", "competencia"], as_index=False).agg({
    "cantidad": "sum",
    "timestamp": "first"  # opcional: puedes tomar la primera fecha o datetime.now()
})

try:
    old_df = pd.read_csv(CSV_EXISTENTE)
    df_final = pd.concat([old_df, df_agrupado], ignore_index=True)
except FileNotFoundError:
    df_final = df_agrupado

# Reordenar columnas antes de guardar
df_final = df_final[["timestamp", "competencia", "categoria", "cantidad"]]

df_final.to_csv(CSV_EXISTENTE, index=False, encoding="utf-8-sig")

print("✅ Datos guardados en", CSV_EXISTENTE)
print(f"🔎 Total MercadoLibre: {total_vehiculos}")
print(f"📊 Suma categorías: {suma_categorias}")
#print(f"➕ Diferencia ajustada en 'Autos sin categoría': {diferencia}")


✅ Datos guardados en competencia.csv
🔎 Total MercadoLibre: 21627
📊 Suma categorías: 22933
➕ Diferencia ajustada en 'Autos sin categoría': -1306


Este nuevo codigo igual es de ML pero acá tomamos los datos de autos qué sean Dealer y particular, estan separados porque se crearon en tiempos distintos.

In [7]:
# requisitos: pip install selenium webdriver-manager pandas beautifulsoup4
import pickle
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import time
import os

# --- Config ---
BASE_URL = "https://autos.mercadolibre.cl/"
ROOT_DOMAIN = "https://mercadolibre.cl/"   # dominio para cargar cookies
COOKIES_PATH = "cookies.pkl"  
CSV_PATH = "mercadolibre_autos.csv"
HEADLESS = False
TIMEOUT = 15

# --- Driver ---
def start_driver(headless=False):
    options = webdriver.ChromeOptions()
    if headless:
        options.add_argument("--headless=new")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--lang=es-CL")
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
    driver.set_window_size(1200, 900)
    return driver

# --- Cargar cookies ---
def load_cookies_to_driver(driver, cookies_path=COOKIES_PATH):
    try:
        with open(cookies_path, "rb") as f:
            cookies = pickle.load(f)
    except Exception as e:
        print(f"⚠️ No pude leer {cookies_path}: {e}")
        return 0

    if not isinstance(cookies, (list, tuple)):
        print("⚠️ Formato inesperado en cookies.pkl")
        return 0

    added = 0
    allowed_fields = {"name", "value", "path", "domain", "secure", "httpOnly", "expiry"}
    for c in cookies:
        if not isinstance(c, dict):
            continue
        if "name" not in c or "value" not in c:
            continue
        cookie = {}
        for k, v in c.items():
            if k not in allowed_fields:
                continue
            if k == "expiry":
                try:
                    cookie["expiry"] = int(v)
                except Exception:
                    continue
            else:
                cookie[k] = v
        try:
            driver.add_cookie(cookie)
            added += 1
        except Exception:
            try:
                cookie_no_domain = {kk: vv for kk, vv in cookie.items() if kk != "domain"}
                driver.add_cookie(cookie_no_domain)
                added += 1
            except:
                pass
    print(f"🍪 Cookies añadidas: {added}")
    return added

# --- Extraer cantidad de autos ---
def extract_count_from_page_source(html):
    soup = BeautifulSoup(html, "html.parser")
    tag = soup.find("span", class_="ui-search-search-result__quantity-results")
    if not tag:
        return None
    text = tag.get_text(strip=True)
    digits = "".join(ch for ch in text if ch.isdigit())
    if digits == "":
        return None
    return int(digits)

# --- Click en filtro por ID, con fallback por texto ---
def click_filter(driver, element_id, fallback_text, label):
    wait = WebDriverWait(driver, TIMEOUT)
    try:
        # intento por ID
        elem = wait.until(EC.element_to_be_clickable((By.ID, element_id)))
    except:
        # fallback por texto
        elems = driver.find_elements(By.XPATH, f"//span[contains(., '{fallback_text}')]/ancestor::label//input[@type='checkbox']")
        if not elems:
            print(f"❌ No se pudo encontrar el filtro '{label}'")
            return None
        elem = elems[0]

    try:
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", elem)
        time.sleep(0.5)
        driver.execute_script("arguments[0].click();", elem)
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "span.ui-search-search-result__quantity-results")))
        time.sleep(1)
        count = extract_count_from_page_source(driver.page_source)
        print(f"✅ {label}: {count}")
        return count
    except Exception as e:
        print(f"❌ Error al clickear '{label}': {e}")
        return None

# --- Main ---
def main():
    driver = start_driver(headless=HEADLESS)
    rows = []
    try:
        # 0) Abrir dominio raíz y cargar cookies
        driver.get(ROOT_DOMAIN)
        WebDriverWait(driver, TIMEOUT).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
        time.sleep(0.8)
        load_cookies_to_driver(driver)
        driver.get(ROOT_DOMAIN)
        time.sleep(0.8)

        # 1) Ir a autos
        driver.get(BASE_URL)
        WebDriverWait(driver, TIMEOUT).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
        time.sleep(1)

        fecha = datetime.now().strftime("%Y-%m-%d")

        # 2) Click DEALER (Usado)
        driver.get(BASE_URL)
        time.sleep(0.8)
        dealer_count = click_filter(driver, "item_condition_highlighted", "Usado", "DEALER")
        rows.append({
            "fecha": fecha,
            "fuente": "MERCADOLIBRE",
            "tipo": "DEALER",
            "cantidad_autos": dealer_count
        })

        # 3) Click PARTICULAR (Vendedor particular)
        driver.get(BASE_URL)
        WebDriverWait(driver, TIMEOUT).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
        time.sleep(1)
        particular_count = click_filter(driver, "seller_type_highlighted", "Vendedor particular", "PARTICULAR")
        rows.append({
            "fecha": fecha,
            "fuente": "MERCADOLIBRE",
            "tipo": "PARTICULAR",
            "cantidad_autos": particular_count
        })

    finally:
        driver.quit()

    # Guardar CSV (append)
    df_new = pd.DataFrame(rows)
    if os.path.exists(CSV_PATH):
        df_old = pd.read_csv(CSV_PATH)
        df_out = pd.concat([df_old, df_new], ignore_index=True)
    else:
        df_out = df_new

    df_out.to_csv(CSV_PATH, index=False, encoding="utf-8-sig")
    print(f"\n📁 Guardado: {CSV_PATH}")
    print(df_new)

if __name__ == "__main__":
    main()


🍪 Cookies añadidas: 24
✅ DEALER: 21625
✅ PARTICULAR: 4763

📁 Guardado: mercadolibre_autos.csv
        fecha        fuente        tipo  cantidad_autos
0  2025-10-06  MERCADOLIBRE      DEALER           21625
1  2025-10-06  MERCADOLIBRE  PARTICULAR            4763


Codigo de extraccion de Yapo.

In [6]:
import datetime 
# ===== CONFIG =====
OUTPUT_CSV = "vehiculos_totales.csv"
WRITE_HEADER = not os.path.exists(OUTPUT_CSV)

options = Options()
options.add_argument("--headless=new")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1280,2000")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 20)

try:
    driver.get(URLYAPO)
    # Esperar a que carguen los labels de filtros
    labels = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.d3-checkbox label")))

    filas = []
    now = datetime.datetime.now().isoformat(timespec="seconds")

    total_vehiculos = 0

    for lb in labels:
        texto = lb.text.strip()
        if not texto:
            continue
        # Extraer número entre paréntesis
        m = re.search(r"\(([\d\.]+)\)", texto)
        numero = int(m.group(1).replace(".", "")) if m else 0

        # Nombre de la categoría
        nombre = re.sub(r"\s*\([\d\.\)]+\)\s*$", "", texto).strip()

        # Sumar total excluyendo "Yo busco"
        if nombre.lower() != "yo busco":
            total_vehiculos += numero

        filas.append({
            "timestamp": now,
            "label_text": texto,
            "name": nombre,
            "count": numero
        })

    # Guardar filas de detalle de filtros (opcional)
    fieldnames = ["timestamp", "label_text", "name", "count"]
    with open("yapo_filters.csv", "a", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(filas)

    print(f"✅ Guardadas {len(filas)} filas en yapo_filters.csv")
    print(f"🔢 Total vehículos (excluyendo 'Yo busco'): {total_vehiculos}")

    # Guardar el total al CSV principal
    file_exists = os.path.exists(OUTPUT_CSV)
    with open(OUTPUT_CSV, "a", newline="", encoding="utf-8") as f:
        fieldnames_main = ["timestamp", "URL_YAPO", "total_vehiculos"]
        
        writer = csv.DictWriter(f, fieldnames=fieldnames_main)
        if not file_exists:
            writer.writeheader()
        writer.writerow({
            "timestamp": now,
            "URL_YAPO": URLYAPO,
            "total_vehiculos": total_vehiculos
        })

    print(f"✅ Total agregado a {OUTPUT_CSV}")

finally:
    driver.quit()


✅ Guardadas 7 filas en yapo_filters.csv
🔢 Total vehículos (excluyendo 'Yo busco'): 47158
✅ Total agregado a vehiculos_totales.csv


A partir de acá el codigo se vuelve un poco monotono, es basicamente correr el codigo y ya. Extraccion MACAL

In [7]:
OUTPUT_CSV = "vehiculos_totales.csv"

# ===== OPCIONES HEADLESS =====
options = Options()
options.add_argument("--headless=new")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1280,2000")

# ===== DRIVER =====
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)
wait = WebDriverWait(driver, 20)


try:
    driver.get(URL_MACAL)
    total_elem = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#label-count-result")))
    total_text = total_elem.text.strip()
    m = re.search(r"(\d+)", total_text)
    total = int(m.group(1)) if m else 0

    now = datetime.datetime.now().isoformat(timespec="seconds")
    filas = [{"timestamp": now, "URL": "MACAL", "total_vehiculos": total}]

    fieldnames = ["timestamp", "URL", "total_vehiculos"]
    with open(OUTPUT_CSV, "a", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writerows(filas)

    print(f"✅ Guardadas {total} VEHICULOS de Macal")

finally:
    driver.quit()

✅ Guardadas 51 VEHICULOS de Macal


In [8]:
OUTPUT_CSV = "vehiculos_totales.csv"

# ===== OPCIONES HEADLESS =====
options = Options()
options.add_argument("--headless=new")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1280,2000")

# ===== DRIVER =====
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)
wait = WebDriverWait(driver, 20)




try:
    driver.get(URL_AUTOSUSADOS)

    total_text = wait.until(EC.presence_of_element_located(
        (By.CSS_SELECTOR, "div.ResultsControls_controlsTitle__dQ7Uq p")
    )).text

    m = re.search(r"de\s+([\d\.]+)", total_text)
    total = int(m.group(1).replace(".", "")) if m else 0

    now = datetime.datetime.now().isoformat(timespec="seconds")
    filas = [{"timestamp": now, "URL": "AUTOSUSADOS", "total_vehiculos": total}]

    fieldnames = ["timestamp", "URL", "total_vehiculos"]
    with open(OUTPUT_CSV, "a", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writerows(filas)

    print(f"✅ Guardado total de AutosUsados: {total}")

finally:
    driver.quit()


✅ Guardado total de AutosUsados: 7741


Este codigo cambia un poco porque esta web no tiene señalado la cantidad de vehiculos registrados, así qué contamos cada vehiculo, y cada tarjetita por pagina, luego vamos a la pagina final y contamos los de la pagina final y calculamos cuantos autos tiene en su plataforma. PLATAFORMA AUTOPIA, a demas este codigo necesita de ayuda de mostrar el navegador porque modo HEADLESS no funciona.

In [9]:
OUTPUT_CSV = "vehiculos_totales.csv"

# ===== OPCIONES HEADLESS =====
options = Options()
#options.add_argument("--headless=new")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1280,2000")

# ===== DRIVER =====
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)
wait = WebDriverWait(driver, 20)


try:
    driver.get(URL_AUTOPIA)
    time.sleep(5)  # esperar a que cargue contenido inicial

    # ===== Contar vehículos =====
    total_vehiculos = 0

    while True:
        # === Scroll para cargar todas las cards de la página actual ===
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        # === Contar cards visibles ===
        cards = driver.find_elements(By.CSS_SELECTOR, "div[id='IdCardVehiculo']")
        total_vehiculos += len(cards)
        print(f"🔎 Página actual: {len(cards)} autos encontrados")

        # === Revisar si hay siguiente página ===
        next_button = driver.find_element(By.CSS_SELECTOR, "li.ant-pagination-next")
        if "ant-pagination-disabled" in next_button.get_attribute("class"):
            break  # llegamos a la última página
        else:
            driver.execute_script("arguments[0].click();", next_button)
            time.sleep(5)  # esperar a que cargue la nueva página

    # ===== Guardar en CSV =====
    now = datetime.datetime.now().isoformat(timespec="seconds")
    fila = {"fecha": now, "url": "AUTOPIA", "total": total_vehiculos}

    try:
        with open(OUTPUT_CSV, "x", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=["fecha", "url", "total"])
            writer.writeheader()
            writer.writerow(fila)
    except FileExistsError:
        with open(OUTPUT_CSV, "a", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=["fecha", "url", "total"])
            writer.writerow(fila)

    print(f"✅ Guardado total de vehículos: {total_vehiculos}")

finally:
    driver.quit()


🔎 Página actual: 18 autos encontrados
🔎 Página actual: 18 autos encontrados
🔎 Página actual: 18 autos encontrados
🔎 Página actual: 18 autos encontrados
🔎 Página actual: 17 autos encontrados
✅ Guardado total de vehículos: 89


En este scraping, no cumple la funcion de web en cambio es una llamada a API, y de ahi obtenemos los autos. LINK o BCI SEGUROS

In [10]:
import requests
import csv
import datetime
import json
# ===== Config =====
OUTPUT_CSV = "vehiculos_totales.csv"
URL_BCI = "https://apivehiculos.bciseguros.cl/api/V1/vehiculo/ObtenerVehiculosPaginados"

# ===== Llamada a la API =====
params = {
    "pagina": 1,
    "cantidad": 10
}

response = requests.post(URL_BCI, json=params)

if response.status_code == 200:
    data = response.json()
    total_vehiculos = data["data"].get("totalRegistros", 0)  # 👈 clave correcta
else:
    print(f"❌ Error en la solicitud: {response.status_code}")
    total_vehiculos = 0

# ===== Guardar en CSV (append si existe) =====
now = datetime.datetime.now().isoformat(timespec="seconds")
fila = {"fecha": now, "url": "BCI", "total": total_vehiculos}

try:
    with open(OUTPUT_CSV, "x", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=["fecha", "url", "total"])
        writer.writeheader()
        writer.writerow(fila)
except FileExistsError:
    with open(OUTPUT_CSV, "a", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=["fecha", "url", "total"])
        writer.writerow(fila)

print(f"✅ Guardado total de vehículos ({total_vehiculos}) en {OUTPUT_CSV}")


✅ Guardado total de vehículos (3911) en vehiculos_totales.csv


In [11]:
import os

OUTPUT_CSV = "vehiculos_totales.csv"

# ===== OPCIONES HEADLESS =====
options = Options()
options.add_argument("--headless=new")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1280,2000")

# ===== DRIVER =====
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)
wait = WebDriverWait(driver, 20)

try:
    driver.get(URL_KAVAK)
    time.sleep(5)  # esperar a que cargue contenido inicial

    # === Capturar el total de vehículos ===
    total_div = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "div.sorting-dropdown_resultsCount__xDWMP"))
    )
    total_text = total_div.text.strip()  # Ej: "803 Resultados"
    total_vehiculos = "".join([c for c in total_text if c.isdigit()])
    print(f"🔢 Total de vehículos en Kavak: {total_vehiculos}")

    # === Guardar en CSV (solo timestamp, url, total_vehiculos) ===
    now = datetime.datetime.now().isoformat(timespec="seconds")
    fila = {
        "timestamp": now,
        "url": "KAVAK",
        "total_vehiculos": total_vehiculos
    }

    fieldnames = ["timestamp", "url", "total_vehiculos"]
    file_exists = os.path.isfile(OUTPUT_CSV)

    with open(OUTPUT_CSV, "a", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        if not file_exists:  # solo la primera vez
            writer.writeheader()
        writer.writerow(fila)

    print(f"✅ Guardado total de vehículos ({total_vehiculos}) en {OUTPUT_CSV}")

finally:
    driver.quit()


🔢 Total de vehículos en Kavak: 678
✅ Guardado total de vehículos (678) en vehiculos_totales.csv


In [12]:
OUTPUT_CSV = "vehiculos_totales.csv"

# ===== OPCIONES HEADLESS =====
options = Options()
options.add_argument("--headless=new")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1280,2000")

# ===== DRIVER =====
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)
wait = WebDriverWait(driver, 20)
try:
    driver.get(URL_CLICAR)
    time.sleep(5)  # esperar a que cargue contenido inicial

    # === Capturar el total de vehículos ===
    total_div = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "span.text-gray-900.font-normal.text-sm.my-2")))
    total_text = total_div.text.strip()  # Ej: "2692 vehículos"
    total_vehiculos = "".join([c for c in total_text if c.isdigit()])  # extraer solo el número
    print(f"🔢 Total de vehículos en Clicar: {total_vehiculos}")

    # === Guardar en CSV ===
    now = datetime.datetime.now().isoformat(timespec="seconds")
    filas = [{
        "timestamp": now,
        "CLICAR_BCI": "CLICAR",
        "total_vehiculos": total_vehiculos
    }]

    fieldnames = ["timestamp", "CLICAR_BCI", "total_vehiculos"]
    with open(OUTPUT_CSV, "a", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        if not file_exists:
            writer.writeheader()
        writer.writerows(filas)

    print(f"✅ Guardado total de vehículos ({total_vehiculos}) en {OUTPUT_CSV}")

finally:
    driver.quit()


🔢 Total de vehículos en Clicar: 2611
✅ Guardado total de vehículos (2611) en vehiculos_totales.csv


In [13]:
OUTPUT_CSV = "vehiculos_totales.csv"

# ===== OPCIONES HEADLESS =====
options = Options()
options.add_argument("--headless=new")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1280,2000")

# ===== DRIVER =====
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)
wait = WebDriverWait(driver, 20)

try:
    driver.get(URLDILY)
    time.sleep(5)  # Esperar a que cargue el contenido inicial

    contador = wait.until(EC.presence_of_element_located(
        (By.XPATH, "//p[contains(text(),'Resultados')]/span")
    ))
    total_vehiculos = contador.text.strip()
    print(f"🔢 Total de vehículos: {total_vehiculos}")


    # Guardar el resultado en un archivo CSV


    fieldnames = ["timestamp", URLDILY, "total_vehiculos"]
    with open(OUTPUT_CSV, "a", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        if not file_exists:
            writer.writeheader()
        writer.writerow({
            "timestamp": now,
            URLDILY: "DILY",
            "total_vehiculos": total_vehiculos
        })

    print(f"✅ Información guardada en {OUTPUT_CSV}")

finally:
    driver.quit()


🔢 Total de vehículos: 2304
✅ Información guardada en vehiculos_totales.csv


Este codigo igual necesita ayuda del navegador porque si no, no consigue los datos.

In [14]:

OUTPUT_CSV = "vehiculos_totales.csv"

# ===== CONFIG =====
options = webdriver.ChromeOptions()
# options.add_argument("--headless")  # opcional para debug



driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 20)

try:
    driver.get(URL_BRAVO)
    time.sleep(5)  # esperar a que cargue la página

    # === Capturar el contador de vehículos ===
    contador = WebDriverWait(driver, 20).until(
    EC.visibility_of_element_located((By.CSS_SELECTOR, "div.fs-14")))
    total_text = contador.text  # Ej: "18 de 121 vehículos"
    print(f"Texto del contador: {total_text}")

    # Extraer solo el número total de vehículos
    total_vehiculos = total_text.split(" de ")[1].split(" ")[0]
    print(f"Total de vehículos disponibles: {total_vehiculos}")

    # Guardar en CSV
    now = datetime.datetime.now().isoformat(timespec="seconds")



    fieldnames = ["timestamp", URL_BRAVO, "BRAVO_AUTO_TOTAL"]
    with open(OUTPUT_CSV, "a", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        if not file_exists:
            writer.writeheader()
        writer.writerow({
            "timestamp": now,
            URL_BRAVO: "BRAVO",
            "BRAVO_AUTO_TOTAL": total_vehiculos
        })
    

    print(f"✅ Guardado en {OUTPUT_CSV}")

finally:
    driver.quit()


Texto del contador: 18 de 86 vehículos
Total de vehículos disponibles: 86
✅ Guardado en vehiculos_totales.csv


In [15]:
# ===== OPCIONES HEADLESS =====
OUTPUT_CSV = "vehiculos_totales.csv"

options = Options()
options.add_argument("--headless=new")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1280,2000")

# ===== DRIVER =====
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)
wait = WebDriverWait(driver, 5)

try:
    driver.get(URL_AUTOCL)
    time.sleep(5)  # esperar a que cargue la página

    # Capturar el <p> que contiene el total de vehículos
    contador = wait.until(EC.presence_of_element_located(
        (By.XPATH, "//p[contains(text(),'Resultados:')]")
    ))

    total_vehiculos = contador.text.split("Resultados:")[1].strip()
    print(f"🔢 Total de vehículos: {total_vehiculos}")

    # Guardar en CSV
    now = datetime.datetime.now().isoformat(timespec="seconds")
    with open(OUTPUT_CSV, "a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow([now, "AUTOCL", total_vehiculos])

    print(f"✅ Información guardada en {OUTPUT_CSV}")

finally:
    driver.quit()


🔢 Total de vehículos: 8898
✅ Información guardada en vehiculos_totales.csv


In [16]:
# ===== OPCIONES HEADLESS =====
OUTPUT_CSV = "vehiculos_totales.csv"

options = Options()
options.add_argument("--headless=new")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1280,2000")

# ===== DRIVER =====
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)
wait = WebDriverWait(driver, 5)


try:
    driver.get(URLVenpu)
    time.sleep(5)  # Espera que cargue JS

    # ===== Selector del total de vehículos =====
    total_element = wait.until(EC.presence_of_element_located(
        (By.CSS_SELECTOR, "h6.text-left")
    ))
    total_text = total_element.text.strip()  # Ej: "587 vehículos encontrados"

    # Extraemos solo el número
    total_vehiculos = total_text.split(" ")[0]
    print(f"🔢 Total de vehículos: {total_vehiculos}")

    # ===== Guardar en CSV =====
    now = datetime.datetime.now().isoformat(timespec="seconds")
    with open(OUTPUT_CSV, "a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        if WRITE_HEADER is None:
            writer.writerow(["timestamp", "total_vehiculos"])
        writer.writerow([now, "Venpu", total_vehiculos])

    print(f"✅ Información guardada en {OUTPUT_CSV}")

finally:
    driver.quit()


🔢 Total de vehículos: 554
✅ Información guardada en vehiculos_totales.csv


In [17]:
import requests
import datetime
import csv

# ===== CONFIG =====
URL_MARKETPLACE = "https://tmp-webapp.fly.dev/api/posts/"
OUTPUT_FILE = "themarketplace_total.csv"

params = {
    "brand": "",
    "model": "",
    "min_price": "",
    "max_price": "",
    "min_year": "",
    "max_year": "",
    "transmission": "",
    "min_mileage": "",
    "max_mileage": "",
    "search_query": "",
    "page": 1
}

# Hacer la petición
resp = requests.get(URL_MARKETPLACE, params=params)
data = resp.json()

# Obtener total de vehículos
total_vehiculos = data.get("count", len(data.get("results", [])))
print(f"🔢 Total de vehículos: {total_vehiculos}")

# Guardar en CSV
now = datetime.datetime.now().isoformat(timespec="seconds")
with open(OUTPUT_CSV, "a", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    if WRITE_HEADER is None:
        writer.writerow(["timestamp", "total_vehiculos"])
    writer.writerow([now, "MARKETPLACE", total_vehiculos])

print(f"✅ Total guardado en {OUTPUT_CSV}")


🔢 Total de vehículos: 461
✅ Total guardado en vehiculos_totales.csv


In [18]:


# ===== OPCIONES HEADLESS =====
OUTPUT_CSV = "vehiculos_totales.csv"

# ===== CONFIG =====
URLLINZE = "https://api.linze.cl/api/v1/publications?page=1&limit=9"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/139.0.0.0 Safari/537.36",
    "Accept": "application/json, text/plain, */*",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Origin": "https://linzecar.cl",
    "Referer": "https://linzecar.cl/"
}
OUTPUT_FILE = "linze_total.csv"

# ===== Petición a la API =====
resp = requests.get(URLLINZE, headers=HEADERS, timeout=30)
data = resp.json()

# ===== Extraer total de vehículos =====
total_vehiculos = data['data']['total']
print(f"🔢 Total de vehículos: {total_vehiculos}")

# Guardar en CSV
now = datetime.datetime.now().isoformat(timespec="seconds")
with open(OUTPUT_CSV, "a", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    if WRITE_HEADER is None:
        writer.writerow(["timestamp", "total_vehiculos"])
    writer.writerow([now, "LINZE", total_vehiculos])



🔢 Total de vehículos: 4795


Esta web, igual avanza a la final y calcula la cantidad.

In [19]:
# ===== OPCIONES HEADLESS =====
OUTPUT_CSV = "vehiculos_totales.csv"

options = Options()
options.add_argument("--headless=new")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1280,2000")

# ===== DRIVER =====
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)
wait = WebDriverWait(driver, 5)


try:
    # ===== Primera página =====
    driver.get(URL_CHILEMOTO)
    time.sleep(3)
    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "span.h4 a.view-details-ics")))
    first_count = len(driver.find_elements(By.CSS_SELECTOR, "span.h4 a.view-details-ics"))
    print(f"📄 Primera página: {first_count} motos")

    # ===== Última página =====
    # Sacamos número de páginas
    try:
        pagination_links = driver.find_elements(By.CSS_SELECTOR, "ul.pagination a.page-link")
        last_page = int(pagination_links[-2].text)  # penúltimo link = última página
    except:
        last_page = 1

    print(f"🔢 Total de páginas: {last_page}")

    if last_page > 1:
        last_url = f"https://www.chilemotos.com/moto/page/{last_page}/"
        driver.get(last_url)
        time.sleep(3)
        wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "span.h4 a.view-details-ics")))
        last_count = len(driver.find_elements(By.CSS_SELECTOR, "span.h4 a.view-details-ics"))
        print(f"📄 Última página: {last_count} motos")
    else:
        last_count = first_count

    # ===== Total aproximado =====
    if last_page > 1:
        totalm = (last_page - 1) * first_count + last_count
    else:
        total = first_count


    now = datetime.datetime.now().isoformat(timespec="seconds")
    with open(OUTPUT_CSV, "a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        if WRITE_HEADER is None:
            writer.writerow(["timestamp", "total_vehiculos"])
        writer.writerow([now, "CHILEMOTO", totalm])
    print(f"✅ Total aproximado de motos: {totalm}")
    # Guardar en CSV


finally:
    driver.quit()


📄 Primera página: 10 motos
🔢 Total de páginas: 84
📄 Última página: 8 motos
✅ Total aproximado de motos: 838


In [20]:
# ===== OPCIONES HEADLESS =====
OUTPUT_CSV = "vehiculos_totales.csv"


HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/139.0.0.0 Safari/537.36"
}

def contar_autos(page):
    """Cuenta autos y vendidos en una página"""
    resp = requests.get(URL_UDACH.format(page), headers=HEADERS)
    soup = BeautifulSoup(resp.text, "html.parser")
    autos = soup.select("div.car-card-body")
    vendidos = [a for a in autos if a.select_one("div.sold-tag")]
    return len(autos), len(vendidos)

# ===== 1. Primera página =====
first_count, first_vendidos = contar_autos(1)
print(f"📄 Página 1: {first_count} autos ({first_vendidos} vendidos)")

# ===== 2. Encontrar última página =====
step = 10
page = step
while True:
    count, _ = contar_autos(page)
    print(f"🔎 Chequeando página {page}: {count} autos")
    if count == 0:
        break
    page += step
    time.sleep(0.2)

# Retroceder hasta encontrar la última con autos
last_page = page - step
while True:
    count, _ = contar_autos(last_page + 1)
    if count == 0:
        break
    last_page += 1
    time.sleep(0.2)

print(f"✅ Última página real: {last_page}")

# ===== 3. Última página =====
last_count, last_vendidos = contar_autos(last_page)
print(f"📄 Página {last_page}: {last_count} autos ({last_vendidos} vendidos)")

# ===== 4. Total aproximado =====
total_autos = (last_page - 1) * first_count + last_count
total_vendidos = (last_page - 1) * first_vendidos + last_vendidos

print(f"🚗 Total aproximado de autos: {total_autos}")
print(f"❌ Total aproximado de vendidos: {total_vendidos}")
total_total = total_autos - total_vendidos
print(f"✅ Total disponibles: {total_total}")


now = datetime.datetime.now().isoformat(timespec="seconds")
with open(OUTPUT_CSV, "a", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    if WRITE_HEADER is None:
        writer.writerow(["timestamp", "total_vehiculos"])
    writer.writerow([now, "UDACH", total_total])
    print(f"✅ Total aproximado: {total_total}")
    # Guardar en CSV


📄 Página 1: 12 autos (0 vendidos)
🔎 Chequeando página 10: 12 autos
🔎 Chequeando página 20: 12 autos
🔎 Chequeando página 30: 12 autos
🔎 Chequeando página 40: 12 autos
🔎 Chequeando página 50: 12 autos
🔎 Chequeando página 60: 12 autos
🔎 Chequeando página 70: 12 autos
🔎 Chequeando página 80: 12 autos
🔎 Chequeando página 90: 12 autos
🔎 Chequeando página 100: 12 autos
🔎 Chequeando página 110: 12 autos
🔎 Chequeando página 120: 12 autos
🔎 Chequeando página 130: 0 autos
✅ Última página real: 129
📄 Página 129: 12 autos (0 vendidos)
🚗 Total aproximado de autos: 1548
❌ Total aproximado de vendidos: 0
✅ Total disponibles: 1548
✅ Total aproximado: 1548


In [22]:
# ===== CONFIG =====
OUTPUT_CSV = "vehiculos_totales.csv"
options = Options()
#options.add_argument("--headless=new")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1280,2000")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 10)

try:
    driver.get(URL_AUTOSCOSMO)
    time.sleep(3)  # esperar a que cargue JS

    # ===== Selector correcto del total de vehículos =====
    total_element = wait.until(EC.presence_of_element_located(
        (By.CSS_SELECTOR, "h2.section__subtitle strong")
    ))
    total_vehiculos = total_element.text.strip()
    print(f"🔢 Total de vehículos: {total_vehiculos}")

    # ===== Guardar en CSV con append seguro =====
    now = datetime.datetime.now().isoformat(timespec="seconds")
    file_exists = os.path.exists(OUTPUT_CSV)

    with open(OUTPUT_CSV, "a", newline="", encoding="utf-8") as f:
        fieldnames = ["timestamp", "URL_AUTOCOSMOS", "total_vehiculos"]
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        if not file_exists:
            writer.writeheader()
        writer.writerow({
            "timestamp": now,
            "URL_AUTOCOSMOS": "AUTOSCOSMO",
            "total_vehiculos": total_vehiculos
        })

    print(f"✅ Información guardada en {OUTPUT_CSV}")

finally:
    driver.quit()


🔢 Total de vehículos: 16903
✅ Información guardada en vehiculos_totales.csv


In [23]:
import pickle

cookies = pickle.load(open("cookies.pkl", "rb"))
print(len(cookies), "cookies guardadas")
for c in cookies[:5]:  # mostrar las 5 primeras
    print(c["name"], "=", c["value"])

24 cookies guardadas
_snoopy = eyJmaW5nZXJwcmludCI6IlNNQWhMcHdESTY2NUpjb09tZ3pldDJ2RTI0U3hlYlRlY29lSkEzMkJTQWwvdFQ4eUdQbG1keHc4aVNqNjBuaDBFdE9LK2taUkNESWtCTC9xTGpBc0RYTU9hOFdqNVNyWHowMzNqYVgvbi9ZdDdJUjVWV3RabzZQTXUzTW9QTFFuek16bEZUQTd4SXR4dDJyd0FSWEI3OVlqUmRNd3BienNVRkN0WXEzZlE4NFRXR3M5QVc0PSIsImtleSI6InB1QlRqUFhib2VNMGwwL1A3cDhSdVJqYTRmZkZMQmd4RnM5cytxUWdLQnZwZXhRSXhsdFJ1L1JmanlhTEpOMC8zbnQzT2ZQOUpLbVhpL2pOSXU2WjBxcGQwbXh3S0JvQyJ9
_mldataSessionId = 8250479a-f7b8-4171-8abf-96bcbef3a0eb
c_ui-navigation = 6.6.146
last_query = 
cp = CL-RM_TUxDQ1BST2NhYjU3


In [24]:
import requests
import pickle
from bs4 import BeautifulSoup
import csv
from datetime import datetime
import os
import re


OUTPUT_CSV = "vehiculos_totales.csv"
URLML = "https://autos.mercadolibre.cl/"

# Cargar cookies
cookies = pickle.load(open("cookies.pkl", "rb"))
session = requests.Session()
for cookie in cookies:
    session.cookies.set(cookie['name'], cookie['value'])

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/114.0.0.0 Safari/537.36",
    "Accept-Language": "es-CL,es;q=0.9,en;q=0.8"
}

# Hacer request
response = session.get(URLML, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

# Extraer total de resultados
total_resultados = soup.select_one("span.ui-search-search-result__quantity-results")
if total_resultados:
    total_vehiculos = total_resultados.get_text(strip=True)
    # Quitar puntos y convertir a int
    total_vehiculos = int(re.sub(r"[^\d]", "", total_vehiculos))
else:
    total_vehiculos = 0

# Timestamp
now = datetime.now().isoformat()

# Guardar en CSV
file_exists = os.path.isfile(OUTPUT_CSV)
with open(OUTPUT_CSV, "a", newline="", encoding="utf-8") as f:
    fieldnames = ["timestamp", "URLML", "total_vehiculos"]
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    if not file_exists:
        writer.writeheader()
    writer.writerow({
        "timestamp": now,
        "URLML": "MERCADOLIBRE",
        "total_vehiculos": total_vehiculos
    })

print(f"✅ Información guardada en {OUTPUT_CSV}: {total_vehiculos} vehículos")

✅ Información guardada en vehiculos_totales.csv: 21625 vehículos
